# PxS inference algorithms

Trying to gather some insights into settings of BayesFusion.

## Preliminaries

### Imports

In [1]:
# Imports
import os
import numpy as np
import pandas as pd
import json
import sys
import pickle as pkl
import warnings

from datetime import datetime
from os.path import dirname

In [2]:
# Custom Imports
root_dir = dirname(dirname(os.getcwd()))
src_dir = os.path.join(root_dir, 'src')
cli_dir = os.path.join(root_dir, 'cli')
cli_cfg_dir = os.path.join(cli_dir, 'config')
sys.path.append(src_dir)

from exp.utils.filesystem import ensure_dir

### Methods

In [3]:
def save_parameters(d, fname, folder=None, fresh=False):
    """
    Function that saves the parameters as a json file.
    """
    
    if folder is not None:
        ensure_dir(folder, empty=fresh)
            
    ext = "json"
    fname = os.path.join(folder, fname) + '.' + ext
    
    with open(fname, 'w') as f:
        json.dump(d, f, indent=4, sort_keys=True)
    return fname

### Global Variables

For all info dataset-related, cf. https://github.com/UCLA-StarAI/Density-Estimation-Datasets. That is the source. 

For the rest this section defines some lists and variables that will recur.

In [4]:
# tiny datasets < 20 variables
tiny_datasets = ['nltcs',
                 'msnbc']

In [5]:
# small datasets: < 101 variables
small_datasets = ['jester',
                  'kdd',
                  'plants',
                  'baudio',
                  'bnetflix']

tiny_small_datasets = tiny_datasets + small_datasets

In [6]:
# middle datasets: (100 < x < 501) variables
medium_datasets = ['accidents',
                   'tretail',
                   'pumsb_star',
                   'dna',
                   'kosarek',
                   'msweb',
                   'book',
                   'tmovie']

In [7]:
# large datasets: 500 < x variables
large_datasets = ['cwebkb',
                  'cr52',
                  'c20ng',
                  'bbc',
                  'voting',
                  'ad']

And some aggregates of those;

In [8]:
tiny_small_datasets = tiny_datasets + small_datasets
medium_large_datasets = medium_datasets + large_datasets
all_datasets = tiny_datasets + small_datasets + medium_datasets + large_datasets

hr_in_s = 60*60
datecode = "{:%y%m%d}".format(datetime.now())

## Local Demo

### Fixed Parameters

Defining some fixed parameters.

In [9]:

cfg_dir = datecode + '-pxs-inf'
cfg_dir = os.path.join(cli_cfg_dir, cfg_dir)
child = 'RunPxSParallel'

In [10]:
cfg_dir

'/cw/dtailocal/Dropbox/Files/KUL/research/codebases/homework/cli/config/190206-pxs-inf'

In [11]:
to_hrs =  0.5 * hr_in_s # 6 mins per query max
folds = [0]             # One fold.

In [12]:
gen_parameters = {"child":                            child,
                  child + ".folds":                   folds,
                  child + ".timeout":                 int(to_hrs)}

qry_parameters = {child + ".qry.keyword":             "it-S",
                  child + ".nb_queries":               100}

prefix = child + ".mod"
mod_parameters = {prefix + ".keyword":             "default",
                  prefix + ".type":                "PxS"}

In [13]:
fixed_parameters = {**gen_parameters, **qry_parameters, **mod_parameters}

### Exploration

Here we define the grid that has to be searched.

Cf. [The documentation](https://support.bayesfusion.com/docs/GeNIe/). For our choices.

Under `Building blocks of genie` --> `Inference Algorithms` --> `Bayesian Network Algorithms`
1. --> `exact algorithms`
2. --> `approximate algorithms`

So Lauritzen is dixit BF the fastest exact one, whereas EPIS should be SOTA approximate. Linear relevance decomp should handle large networks better, for the rest is like Lauritzen.

In [14]:
idx = 4

In [15]:
exp_parameters = {'idx':             idx,
                  'explore.dataset': all_datasets,
                  'explore.PxS.inference_algorithm':   ['lauritzen',
                                                        'EPIS_sampling',
                                                        'linear_relevance_decomposition']}

In [16]:
fname = str(exp_parameters['idx']) +'_' +  'pxs-inf'
save_parameters({**fixed_parameters, **exp_parameters}, fname, folder=cfg_dir, fresh=True)

'/cw/dtailocal/Dropbox/Files/KUL/research/codebases/homework/cli/config/190206-pxs-inf/1_pxs-inf.json'